<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2020</h1>
<h1 style="font-size: 250%;">Problem Set #3 </h1>
<h1>Issued Tuesday, 2/18/2020; Due Tuesday, 3/3/2020</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of 70.71% for up to two class meetings after the due date. Feel free to get help from others, but **the work you submit in should be your own.**

**Warning:** This notebook has been annotated with metadata so that it can be uploaded to the grading system. It is very important that you enter your answers in the provided cells. You can add extra cells to explore approaches, but only the provided cell can and will be graded. Thus, if you delete a cell and add a replacement, there is a possiblity that your problem will not be graded. If you ever need to start over, you should download a new version of the problem set and transfer your solutions to it.

In [1]:
# Replace the following string values with the requested information
class Student:
    first = "Komal"
    last = "Essarani"
    onyen = "kessara0"
    pid = "730235611"

You will need a set of 15 base-pair <a href="http://csbio.unc.edu/mcmillan/Comp555S20/data/reads.fa" download="reads.fa">simulated reads</a> from a genome that will be used for assembly.
The cell below provides various functions and imports necessary for this problem set. Do not import any addtional packages. Also, make sure that you ***run the following cell***.

In [5]:
import itertools
import math
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into two lists. One of 
        headers and a second list of sequences.
        The ith index of each list correspond."""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'r')
    else:
        fp = open(filename, 'r')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

class Graph:
    def __init__(self, vlist=[]):
        """ Initialize a Graph with an optional vertex list """
        self.index = {v:i for i,v in enumerate(vlist)}
        self.vertex = {i:v for i,v in enumerate(vlist)}
        self.edge = []
        self.edgelabel = []
    def addVertex(self, label):
        """ Add a labeled vertex to the graph """
        index = len(self.index)
        self.index[label] = index
        self.vertex[index] = label
    def addEdge(self, vsrc, vdst, label='', repeats=True):
        """ Add a directed edge to the graph, with an optional label. 
        Repeated edges are distinct, unless repeats is set to False. """
        e = (self.index[vsrc], self.index[vdst])
        if (repeats) or (e not in self.edge):
            self.edge.append(e)
            self.edgelabel.append(label)
    def hamiltonianPath(self):
        """ A Brute-force method for finding a Hamiltonian Path. 
        Basically, all possible N! paths are enumerated and checked
        for edges. Since edges can be reused there are no distictions
        made for *which* version of a repeated edge. """
        for path in itertools.permutations(sorted(self.index.values())):
            for i in range(len(path)-1):
                if ((path[i],path[i+1]) not in self.edge):
                    break
            else:
                return [self.vertex[i] for i in path]
        return []
    def SearchTree(self, path, verticesLeft):
        """ A recursive Branch-and-Bound Hamiltonian Path search. 
        Paths are extended one node at a time using only available
        edges from the graph. """
        if (len(verticesLeft) == 0):
            self.PathV2result = [self.vertex[i] for i in path]
            return True
        for v in verticesLeft:
            if (len(path) == 0) or ((path[-1],v) in self.edge):
                if self.SearchTree(path+[v], [r for r in verticesLeft if r != v]):
                    return True
        return False
    def hamiltonianPathV2(self):
        """ A wrapper function for invoking the Branch-and-Bound 
        Hamiltonian Path search. """
        self.PathV2result = []
        self.SearchTree([],sorted(self.index.values()))                
        return self.PathV2result
    def degrees(self):
        """ Returns two dictionaries with the inDegree and outDegree
        of each node from the graph. """
        inDegree = {}
        outDegree = {}
        for src, dst in self.edge:
            outDegree[src] = outDegree.get(src, 0) + 1
            inDegree[dst] = inDegree.get(dst, 0) + 1
        return inDegree, outDegree
    def verifyAndGetStart(self):
        inDegree, outDegree = self.degrees()
        start = 0
        end = 0
        for vert in self.vertex.keys():
            ins = inDegree.get(vert,0)
            outs = outDegree.get(vert,0)
            if (ins == outs):
                continue
            elif (ins - outs == 1):
                end = vert
            elif (outs - ins == 1):
                start = vert
            else:
                start, end = -1, -1
                break
        if (start >= 0) and (end >= 0):
            return start
        else:
            return -1
    def eulerianPath(self):
        graph = [(src,dst) for src,dst in self.edge]
        currentVertex = self.verifyAndGetStart()
        path = [currentVertex]
        # "next" is where vertices get inserted into our tour
        # it starts at the end (i.e. it is the same as appending),
        # but later "side-trips" will insert in the middle
        next = 1
        while len(graph) > 0:
            for edge in graph:
                if (edge[0] == currentVertex):
                    currentVertex = edge[1]
                    graph.remove(edge)
                    path.insert(next, currentVertex)
                    next += 1
                    break
            else:
                for edge in graph:
                    try:
                        next = path.index(edge[0]) + 1
                        currentVertex = edge[0]
                        break
                    except ValueError:
                        continue
                else:
                    print("There is no path!")
                    return False
        return path
    def eulerEdges(self, path):
        edgeId = {}
        for i in range(len(self.edge)):
            edgeId[self.edge[i]] = edgeId.get(self.edge[i], []) + [i]
        edgeList = []
        for i in range(len(path)-1):
            edgeList.append(self.edgelabel[edgeId[path[i],path[i+1]].pop()])            
        return edgeList
    def render(self, highlightPath=[]):
        """ Outputs a version of the graph that can be rendered
        using graphviz tools (http://www.graphviz.org/)."""
        edgeId = {}
        for i in range(len(self.edge)):
            edgeId[self.edge[i]] = edgeId.get(self.edge[i], []) + [i]
        edgeSet = set()
        for i in range(len(highlightPath)-1):
            src = self.index[highlightPath[i]]
            dst = self.index[highlightPath[i+1]]
            edgeSet.add(edgeId[src,dst].pop())
        result = ''
        result += 'digraph {\n'
        result += '   graph [nodesep=2, size="10,10"];\n'
        for index, label in self.vertex.iteritems():
            result += '    N%d [shape="box", style="rounded", label="%s"];\n' % (index, label)
        for i, e in enumerate(self.edge):
            src, dst = e
            result += '    N%d -> N%d' % (src, dst)
            label = self.edgelabel[i]
            if (len(label) > 0):
                if (i in edgeSet):
                    result += ' [label="%s", penwidth=3.0]' % (label)
                else:
                    result += ' [label="%s"]' % (label)
            elif (i in edgeSet):
                result += ' [penwidth=3.0]'                
            result += ';\n'                
        result += '    overlap=false;\n'
        result += '}\n'
        return result

**Problem #1:** Construct two minimal DNA superstrings that include every 4-base k-mer and begins with "ACAT". Find the first using using hamiltonianPathV2() and time it. Find the second using eulerianPath() and time it. Your answer should include both strings and their timings. Are your minimal superstrings identical?

In [7]:
# This cell is provided for work on problem 1. It will not be graded.

# find where string starts with ACAT
def bruteForcePatternMatching(p, t):
    locations = []
    for i in range(0, len(t)-len(p)+1):
        if t[i:i+len(p)] == p:
            locations.append(i)
    return locations

# ----------------------------------------------------------------------------------------------------------------------- #

combo = [''.join(t) for t in itertools.product('ATGC', repeat=4)]
nodes = sorted(set([c[:-1] for c in combo] + [c[1:] for c in combo]))
e_g = Graph(nodes)
h_g = Graph(combo)

# hamiltonian path stuff
print("HamiltonianPath stuff:")
for c in combo:
    h_g.addEdge(c,c[1:]+'A')
    h_g.addEdge(c,c[1:]+'C')
    h_g.addEdge(c,c[1:]+'G')
    h_g.addEdge(c,c[1:]+'T')
%time h_path = h_g.hamiltonianPathV2()
h_superstring = h_path[0] + ''.join([h_path[i][3] for i in range(1,len(h_path))])
h_loc = bruteForcePatternMatching("ACAT", h_superstring)
print("hamiltonianPathV2 superstring:", h_superstring[h_loc[0]:]+h_superstring[0:h_loc[0]:])

# eulerian path stuff
for c in combo:
    e_g.addEdge(c[:-1],c[1:],c)

print("\nEulerianPath stuff:")
%time path = e_g.eulerianPath()
edges = e_g.eulerEdges(path)
e_superstring = (edges[0] + ''.join([edges[i][-1] for i in range(1,len(edges))]))

location = bruteForcePatternMatching("ACAT", e_superstring)
print("eulerianPath superstring:", e_superstring[location[0]:]+e_superstring[0:location[0]:])

HamiltonianPath stuff:
Wall time: 310 ms
hamiltonianPathV2 superstring: ACATTTATTGATTCATGTATGGATGCATCTATCGATCCAGAGACAGTTAGTGAGTCAGGTAGGGAGGCAGCTAGCGAGCCACACTTACTGACTCACGTACGGACGCACCTACCGACCCTTTTGTTTCTTGGTTGCTTCGTTCCTGTGTCTGGGTGGCTGCGTGCCTCTCGGTCGCTCCGTCCCGGGGCGGCCGCGCCCCAAAAAAATAAAGAAACAATTAATGAATCAAGTAAGGAAGCAACTAACGAACCATATAGAT

EulerianPath stuff:
Wall time: 6.88 ms
eulerianPath superstring: ACATAAAGTTAGTGAGTCAGTAAGGTAGGGAGGCAGGAAGCTAGCGAGCCAGCAAGAAACTTACTGACTCACTAACGTACGGACGCACGAACCTACCGACCCACCAACAAAAAAATTTTGGGGCCCCGCCGGCGCGGGTGGCTGGTTGCGTGCCTGCTTGTGTCTCTGTTTCGGTCGCTCGTTCCGTCCCTCCTTCTTTATTGATTCATTAATGTATGGATGCATGAATCTATCGATCCATCAATATAGAGACACAGAT


**Problem #2:** A file of 25-mers simulating short reads from a genome can be <a href="http://csbio.unc.edu/mcmillan/Comp555S20/data/reads.fa" download="reads.fa">downloaded here</a>. How many distinct nodes appear in the De Bruijn graph that represents these 19-mers as edges? How many nodes are semi-balenced? How many nodes are balanced? How many are balanced with both in-degrees and out-degrees equal to 1?

In [8]:
# This cell is provided for your own use in answering problem 4. It will not be graded.
headers, reads2 = loadFasta("reads2.fa")
reads2 = [r[1:] for r in reads2]
nodes = set([r[:-1] for r in reads2] + [r[1:] for r in reads2])

graph = Graph(nodes)
for r in reads2:
    graph.addEdge(r[:-1], r[1:], r)

inDeg, outDeg = graph.degrees()
balanceCount = 0
oneBalance = 0
semiBalance = 0

for v in graph.vertex:
    inD = inDeg.get(v)
    outD = outDeg.get(v)
    if(inD == None):
        inD = 0
    if(outD == None):
        outD = 0
    if(abs(outD-inD) == 1):
        semiBalance += 1
    if(inD == outD):
        balanceCount += 1
        if(inD == 1 and outD == 1):
            oneBalance += 1
            
distinctCount = len(graph.vertex)

print("distinct nodes:", distinctCount)
print("balanced nodes:", balanceCount) 
print("balanced nodes with indegree and outdegree == 1:", oneBalance)
print("semi-balanced nodes:", semiBalance)

distinct nodes: 9966
balanced nodes: 9964
balanced nodes with indegree and outdegree == 1: 9946
semi-balanced nodes: 2


**Problem #3:** What is the length of the Eulerian path that can be constructed in the De Bruijn graph described in Problem #2? It should be an assembled contig from the genome <a href="http://csbio.unc.edu/mcmillan/Comp555S20/data/Chromo.fa" download="Chromo.fa">Chromo.fa</a>. How long did it take to assemble it using Jupyter %time? Find the starting position of this assembled sequence in that genome.

In [9]:
# This cell is provided for your own use in answering problem 3. It will not be graded.
%timeit path2 = graph.eulerianPath()
path2 = graph.eulerianPath()
print("length of path:", len(path2))

edges = graph.eulerEdges(path2)
superstring = edges[0] + ''.join([edges[i][-1] for i in range(1,len(edges))])
header, sequence = loadFasta("Chromo.fa")
sequence = sequence[0][1:]
start_position = sequence.find(superstring)
print("the starting position:", start_position)

2.77 s ± 234 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
length of path: 9987
the starting position: 13999999


**Problem #4:** Construct a suffix array of the substring from <a href="http://csbio.unc.edu/mcmillan/Comp555S20/data/Chromo.fa" download="Chromo.fa">Chromo.fa</a> from positions 30000000 to 30050000 with a '$' appended. Use it to find all the positions of the substring "CATCAT" within this substring (include the offset of 30000000), and their interval within the suffix array.

In [10]:
# This cell is provided for your own use in answering problem 4. It will not be graded.
header, sequence = loadFasta("Chromo.fa")
substring = sequence[0][30000000:30050000] +'$'

def suffixArray(string):
    return [i for i in sorted(range(len(string)), key=lambda x: string[x:])]

sa = suffixArray(substring)

def findFirst(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while (lo < hi):
        middle = (lo+hi)//2
        if text[suffixarray[middle]:] < pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo

first = findFirst("CATCAT", substring, sa)

def findLast(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while (lo < hi):
        middle = (lo+hi)//2
        if text[suffixarray[middle]:suffixarray[middle]+len(pattern)] <= pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo

last = findLast("CATCAT", substring, sa)
catIndex = []

for suffix in sa[first:last]:     
    catIndex.append(suffix+30000000)
    print("%3d: %s" % (suffix, substring[suffix:]), "\n")

print("CATCAT appears", last - first, "times at these positions", catIndex)
print("the interval between which CATCAT appears is", first, "to", last)

10678: ...$ 

8273: ...$ 

4059: ...$ 

12979: ...$ 

23264: ...$ 

25965: ...$ 

26400: ...$ 

38568: CATCATGCCATTCCCATGGTAACCCACACCAGTGTGACATCTTATAAGCTTCATCCTCAGTCCCTCCCTACCAATGTGGTTTGGATCGCATCTTAATGATTGCATCATTAAGATCACTTAATGGCTTTGTTTACGAAGCAGCCACTATGATGCCGCCAGGTCCCACCCAGGCCTAGTGCTTCATGCACAAACATTACGTCACTTACTCTTCCTACCACCTTGCCAAGGGCTTGCTGTTATCCTCATCTTAAAGATGGGGCAATCACAACCCAGAGATGTTCATTAACTGGCTCATGGCCTCCCAGTTAGGAAACGGTAGCATAAAGGAGAAAATATCTCTATTAGTCTGAGTCCAAAGTGGAGATCAAATTCTGCTTACTGTCTGTGGTCAGAACCAGGTAGACCTGAGAAGTCTCAACAAGCTGCCATGAACCAGTTCACTCTCTCCCTTGTGTACTCTGTCTCCATGACTATGCTTCTCCTCTGGAGCCTCATTTATACCTCAGGACTTCACTATGTCCTGACCCTGGTTTACTTTGCTAAGATACTCATATCCTTCTTGACTTAGTTAATTCCTCTCTCCCAGGGACTGAAACTAGTCCCTTTAGCTGCTAGATAGAATCTTGTTTTCTACTGGGCAGATGTTCCTAGAACCACGAACCACCCTTCTGCCTTGTCTATGTACATGAACTCTACCTGGAAATGGCTGTGTGATATTGATACAAACTCACTCTCTCCTTCCTCAGCTGGCTTAAACGCACATATGTCACCCCATATTGGGACGGTTACATATAATGAATGTATATGTGTGGTATTTGCCCTCGGCATCTGCTCTTGACCAATTCACTCTAGGCTTTATCTTTTTTTTTCTCCCTAGTTTCAAGATCTCCATTGGAA

**Problem #5:** Use your suffix array from Problem #4 to construct a BWT of the specified substring. What is the longest run of uninterrupted "A"s, "C"s, "G"s, and "T"s in this BWT. 

In [16]:
# This cell is provided for your own use in answering problem 5. It will not be graded.
def BWTfromSuffixArray(text, suffixarray):
    return ''.join(text[i-1] for i in suffixarray)

bwt = BWTfromSuffixArray(substring, sa)

def count_and_reduce(s, a):
    num = 0
    maxnum = 0
    out = ''
    for c in s:
        if c == a:
            num += 1
            maxnum = max(num, maxnum)
        else:
            num = 0
        if num <= 2:
            out += c

    return maxnum

print("max consecutive num of A's:", count_and_reduce(bwt, "A"))
print("max consecutive num of C's:", count_and_reduce(bwt, "C"))
print("max consecutive num of G's:", count_and_reduce(bwt, "G"))
print("max consecutive num of T's:", count_and_reduce(bwt, "T"))

max consecutive num of A's: 23
max consecutive num of C's: 44
max consecutive num of G's: 23
max consecutive num of T's: 36


---

## Instructions for submitting your problem set

When you are ready to submit a version of your problem set, follow the instructions below.

1. Press [Save and Checkpoint] on the *File* menu of your Jupyter notebook.
2. Press the link below, which will take you to a website for submitting your problem set.
3. Choose the ***correct problem set number*** from the pull-down, else you might overwrite an earlier submission.
4. Enter in your onyen and PID in the form provided, then upload your submission.

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set

**Instructions for resubmissions:**

1. You may resubmit as many times as you like before the deadline. 
2. Resubmissions *always* overwrite any earlier submissions. 
3. If you resubmit after the due date, you will be warned of any penalties. 
4. Problem sets will not be regraded.